In [284]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import glob
import numpy as np
import pandas as pd
import random
import time
import os
import sys
import argparse
import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
from torch.utils.data import random_split

In [285]:
# !gdown --fuzzy https://drive.google.com/file/d/1F9uinZY-eG4x9dNsUOOtAAZMYq6p945U/view?usp=drive_link
# !unzip -qq "ASL-Sensor-Dataglove-Dataset.zip" -d glove_data
# !echo "Unzip successfully"

In [286]:
class MCDCNN(nn.Module):
    def __init__(self, input_channels, num_classes, sequence_length):
        super(MCDCNN, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(input_channels, 64, kernel_size=5, padding=2)
        self.conv3 = nn.Conv1d(input_channels, 64, kernel_size=7, padding=3)
        self.pool = nn.MaxPool1d(2)
        self.fc1 = nn.Linear(64 * 3 * (sequence_length // 2), 100)
        self.fc2 = nn.Linear(100, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.transpose(1, 2)  # Change shape from [batch_size, sequence_length, input_channels] to [batch_size, input_channels, sequence_length]
        x1 = self.relu(self.pool(self.conv1(x)))
        x2 = self.relu(self.pool(self.conv2(x)))
        x3 = self.relu(self.pool(self.conv3(x)))
        x = torch.cat((x1, x2, x3), dim=1)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [287]:
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)
        
        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()
        
    def forward(self, x):
        identity = x.clone()
        
        x = self.relu(self.batch_norm1(self.conv1(x)))
        
        x = self.relu(self.batch_norm2(self.conv2(x)))
        
        x = self.conv3(x)
        x = self.batch_norm3(x)
        
        #downsample if needed
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        #add identity
        x+=identity
        x=self.relu(x)
        
        return x

class Block(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Block, self).__init__()
       

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)

        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()

    def forward(self, x):
      identity = x.clone()

      x = self.relu(self.batch_norm2(self.conv1(x)))
      x = self.batch_norm2(self.conv2(x))

      if self.i_downsample is not None:
          identity = self.i_downsample(identity)
      print(x.shape)
      print(identity.shape)
      x += identity
      x = self.relu(x)
      return x


        
        
class ResNet(nn.Module):
    def __init__(self, ResBlock, layer_list, num_classes, num_channels=3):
        super(ResNet, self).__init__()
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(ResBlock, layer_list[0], planes=64)
        self.layer2 = self._make_layer(ResBlock, layer_list[1], planes=128, stride=2)
        self.layer3 = self._make_layer(ResBlock, layer_list[2], planes=256, stride=2)
        self.layer4 = self._make_layer(ResBlock, layer_list[3], planes=512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*ResBlock.expansion, num_classes)
        
    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        
    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []
        
        if stride != 1 or self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )
            
        layers.append(ResBlock(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion
        
        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))
            
        return nn.Sequential(*layers)

        
        
def ResNet50(num_classes, channels=1):
    return ResNet(Bottleneck, [3,4,6,3], num_classes, channels)
    
def ResNet101(num_classes, channels=3):
    return ResNet(Bottleneck, [3,4,23,3], num_classes, channels)

def ResNet152(num_classes, channels=3):
    return ResNet(Bottleneck, [3,8,36,3], num_classes, channels)

In [288]:
class TimeSeriesDataset(Dataset):
    def __init__(self, root_dir, feature_names=[], channel_size=1):
        self.data = []
        self.labels = []
        self.label_map = {}
        self.feature_names = feature_names
        self.channel_size = channel_size
        self.load_data(root_dir)

    def load_data(self, root_dir):
        label_idx = 0
        for individual_dir in sorted(os.listdir(root_dir)):
            individual_path = os.path.join(root_dir, individual_dir)
            for class_dir in sorted(os.listdir(individual_path)):
                class_path = os.path.join(individual_path, class_dir)
                if os.path.isdir(class_path):
                    for file in glob.glob(os.path.join(class_path, "*.csv")):
                        df = pd.read_csv(file, usecols=self.feature_names)
                        data = df.values.astype(np.float32)
                        data = data[np.newaxis, :]  
                        self.data.append(data)

                        
                        class_name = os.path.splitext(os.path.basename(file))[0]
                        if class_name not in self.label_map:
                            self.label_map[class_name] = label_idx
                            label_idx += 1
                        self.labels.append(self.label_map[class_name])

        self.data = np.array(self.data)
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx]), torch.tensor(self.labels[idx])

In [289]:
root_dir = 'glove_data'
feature_names = [
    "flex_1", "flex_2", "flex_3", "flex_4", "flex_5",
    "GYRx", "GYRy", "GYRz",
    "ACCx", "ACCy", "ACCz"
]


dataset = TimeSeriesDataset(root_dir, feature_names)

dataset_size = len(dataset)
train_size = int(0.7 * dataset_size)  # 80% for training
val_size = dataset_size - train_size  # 20% for validation

# Split the dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for both sets
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [290]:
# Hyperparameters
input_channels = len(feature_names)  # Adjust based on your feature names
num_classes = len(dataset.label_map)  # Number of unique classes
learning_rate = 0.002
num_epochs = 20

In [291]:
# # Initialize model, loss function and optimizer
# model = MCDCNN(input_channels=input_channels, num_classes=num_classes, sequence_length=sequence_length).to('cuda')


# # Training loop
# for epoch in range(num_epochs):
#     model.train()  # Set the model to training mode
#     running_loss = 0.0
#     for inputs, labels in train_dataloader:
#         # Verify the input shape before passing it to the model
#         inputs, labels = inputs, labels
#         inputs, labels = inputs.to('cuda'), labels.to('cuda')

#         # Forward pass
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
    
#     # Calculate and print the average training loss for this epoch
#     avg_train_loss = running_loss / len(train_dataloader)
#     print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}')

#     # Validation phase
#     model.eval()  # Set the model to evaluation mode
#     val_loss = 0.0
#     with torch.no_grad():  # Disable gradient computation
#         for val_inputs, val_labels in val_dataloader:
#             val_inputs, val_labels = val_inputs, val_labels

#             val_inputs, val_labels = val_inputs.to('cuda'), val_labels.to('cuda')

#             # Forward pass
#             val_outputs = model(val_inputs)
#             val_loss += criterion(val_outputs, val_labels).item()
    
#     # Calculate and print the average validation loss for this epoch
#     avg_val_loss = val_loss / len(val_dataloader)
#     print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}')

# print("Training complete")

In [292]:
net = ResNet50(num_classes=num_classes).to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [293]:
for epoch in range(num_epochs):
    net.train()  # Set the model to training mode
    running_loss = 0.0
    for inputs, labels in train_dataloader:
        # Verify the input shape before passing it to the model
        inputs, labels = inputs, labels
        inputs, labels = inputs.to('cuda'), labels.to('cuda')

        # Forward pass
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    # Calculate and print the average training loss for this epoch
    avg_train_loss = running_loss / len(train_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}')

    # Validation phase
    net.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():  # Disable gradient computation
        for val_inputs, val_labels in val_dataloader:
            val_inputs, val_labels = val_inputs, val_labels
            val_inputs, val_labels = val_inputs.to('cuda'), val_labels.to('cuda')

            # Forward pass
            val_outputs = net(val_inputs)
            val_loss += criterion(val_outputs, val_labels).item()
    
    # Calculate and print the average validation loss for this epoch
    avg_val_loss = val_loss / len(val_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}')

print("Training complete")

Epoch [1/20], Train Loss: 4.5607


/home/orange/miniconda3/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [1/20], Validation Loss: 5.1575
Epoch [2/20], Train Loss: 3.2060
Epoch [2/20], Validation Loss: 32.2451
Epoch [3/20], Train Loss: 3.0317
Epoch [3/20], Validation Loss: 3663.5059
Epoch [4/20], Train Loss: 2.7715
Epoch [4/20], Validation Loss: 36.1914
Epoch [5/20], Train Loss: 2.6027
Epoch [5/20], Validation Loss: 2.6083
Epoch [6/20], Train Loss: 2.6595
Epoch [6/20], Validation Loss: 39.5837
Epoch [7/20], Train Loss: 2.5130
Epoch [7/20], Validation Loss: 2.2252
Epoch [8/20], Train Loss: 2.1933
Epoch [8/20], Validation Loss: 6.5273
Epoch [9/20], Train Loss: 2.1118
Epoch [9/20], Validation Loss: 2.6939
Epoch [10/20], Train Loss: 1.9335
Epoch [10/20], Validation Loss: 2.0803
Epoch [11/20], Train Loss: 1.9528
Epoch [11/20], Validation Loss: 1.7762
Epoch [12/20], Train Loss: 1.7648
Epoch [12/20], Validation Loss: 1.7625
Epoch [13/20], Train Loss: 1.6416
Epoch [13/20], Validation Loss: 2.8773
Epoch [14/20], Train Loss: 1.7387
Epoch [14/20], Validation Loss: 24.2057
Epoch [15/20], Train L